In [1]:
import lsst.meas.extensions.gaap
import lsst.afw.table as afwTable
import lsst.afw.detection as afwDetection
import lsst.afw.geom as afwGeom
import lsst.meas.base as measBase
from lsst.ip.diffim import ModelPsfMatchTask
import galsim
import sys
sys.path.append('/home/kannawad/repo/meas_extensions_gaap/tests/')
from test_gaap import GaapFluxTestCase

In [8]:
gaapConfig = lsst.meas.extensions.gaap.GaapFluxConfig()
gaapConfig.scaleByFwhm = True
schema = afwTable.SourceTable.makeMinimalSchema()
cat = afwTable.SourceCatalog(schema)
record = cat.addNew()
gaapPlugin = lsst.meas.extensions.gaap.GaapFluxPlugin(gaapConfig, 'ext_testGaap_GaapFlux', schema, None)
seeing = exposure.getPsf().getSigma()
sF = 1.15

NameError: name 'exposure' is not defined

In [46]:
gaapPlugin._generic.measure(record, exposure, exposure.getBBox().getCenter())

GaapConvolutionError: Problematic scaling factors = 1.15 Errors: AttributeError("'NoneType' object has no attribute 'getBBox'")

In [9]:
testCase = lsst.utils.tests.TestCase()

In [5]:
testCase = GaapFluxTestCase()
algorithm, schema = testCase.makeAlgorithm(gaapConfig)
testCase.setUp()
exposure, catalog = testCase.dataset.realize(noise=0, schema=schema)
record = catalog[0]
#algorithm.measure(record, exposure)

seeing = exposure.getPsf().getSigma()
sF = 1.15
sigma = (2**0.5)*sF*seeing
targetSigma = sF*seeing
targetPsf = afwDetection.GaussianPsf(65, 65, targetSigma)
result, bbox = algorithm._generic.convolve(exposure, targetPsf, record)
convolved = result.psfMatchedExposure[bbox]

center = bbox.getCenter()
aperSigma2 = sigma**2 - targetSigma**2
aperShape = afwGeom.Quadrupole(aperSigma2, aperSigma2, 0.0)
fluxResult = measBase.SdssShapeAlgorithm.computeFixedMomentsFlux(convolved.getMaskedImage(), aperShape, center)
fluxScaling = 0.5*sigma**2/aperSigma2
print(fluxResult.instFlux, fluxScaling, fluxResult.instFlux*fluxScaling)

999.9011269199837 1.0 999.9011269199837


In [6]:
galsim.hsm.FindAdaptiveMom(galsim.Image(exposure.image.array))

galsim.hsm.ShapeData(image_bounds=galsim.BoundsI(xmin=1, xmax=240, ymin=1, ymax=1600), moments_status=0, observed_shape=galsim.Shear((2.7755575615628914e-16-5.12247186678009e-18j)), moments_sigma=2.0, moments_amp=1000.0, moments_centroid=galsim.PositionD(x=121.0, y=801.0), moments_rho4=1.9999999952057053, moments_n_iter=11, correction_status=-10)

In [7]:
result.psfMatchedExposure.image.array[10:-10,10:-10]

array([[ 0.03333214,  0.09427542,  0.22071725,  0.42773694,  0.6861514 ,
         0.91109943,  1.0014157 ,  0.9110995 ,  0.6861513 ,  0.42773694,
         0.22071725,  0.09427541,  0.03333214],
       [ 0.09427542,  0.2666449 ,  0.6242677 ,  1.2097934 ,  1.9406822 ,
         2.5769157 ,  2.8323627 ,  2.576916  ,  1.940682  ,  1.2097933 ,
         0.6242677 ,  0.26664487,  0.09427543],
       [ 0.22071725,  0.62426764,  1.4615322 ,  2.8323617 ,  4.543514  ,
         6.033061  ,  6.631112  ,  6.0330615 ,  4.543514  ,  2.8323617 ,
         1.4615322 ,  0.62426764,  0.22071727],
       [ 0.4277369 ,  1.2097936 ,  2.8323622 ,  5.488948  ,  8.805059  ,
        11.691712  , 12.850699  , 11.691713  ,  8.805059  ,  5.488948  ,
         2.8323622 ,  1.2097936 ,  0.42773697],
       [ 0.6861514 ,  1.9406822 ,  4.5435147 ,  8.805058  , 14.124577  ,
        18.75518   , 20.614365  , 18.75518   , 14.124577  ,  8.805058  ,
         4.5435147 ,  1.9406823 ,  0.6861514 ],
       [ 0.91109943,  2.576916

In [8]:
galsim.hsm.FindAdaptiveMom(galsim.Image(result.psfMatchedExposure[bbox].image.array))

galsim.hsm.ShapeData(image_bounds=galsim.BoundsI(xmin=1, xmax=13, ymin=1, ymax=13), moments_status=0, observed_shape=galsim.Shear((9.614146811998127e-09-4.398987840659175e-09j)), moments_sigma=2.297983169555664, moments_amp=999.0247192382812, moments_centroid=galsim.PositionD(x=6.999999963805932, y=7.000000057522636), moments_rho4=1.9939034232790758, moments_n_iter=12, correction_status=-10)

In [9]:
gaapConfig = lsst.meas.extensions.gaap.GaapFluxConfig()
gaapConfig.modelPsfMatch.kernel.active.alardDegGauss = [0]
gaapConfig.scaleByFwhm = True
gaapConfig.modelPsfMatch.kernel.active.alardGaussBeta = (sigma**2-targetSigma**2)**0.5

In [10]:
import lsst.daf.butler as dafButler

In [11]:
butler = dafButler.Butler('/repo/main', collections='HSC/runs/RC2/w_2021_14/DM-29670', instrument='HSC')

In [12]:
butler.get('deepCoadd', {'tract':9615, 'patch':54, 'band':'r'})

In [36]:
from scipy.optimize import curve_fit

In [37]:
curve_fit?

In [38]:
testCase.dataset.realize?

In [2]:
table = afwTable.SourceCatalog(schema)

NameError: name 'schema' is not defined

In [3]:
afwTable.io.setPreppedRowsFactor(128)
table = afwTable.SourceCatalog(schema)
table

<class 'lsst.afw.table.SourceCatalog'>
 id coord_ra ... ext_testGaap_GaapFlux_flag_edge
      rad    ...                                
--- -------- ... -------------------------------

In [ ]:
table[0] = record

In [1]:
table.

Object `afwTable.SourceCatalog` not found.


In [ ]:
table = afwTable.SourceCatalog

In [ ]:
table = afwTable.SourceCatalog

In [ ]:
table = afwTable.SourceCatalog.reserve

In [4]:
table = afwTable.SourceCatalog(schema)

In [17]:
!pwd

/home/kannawad/jhome/GAaP_analysis/GAaP_analysis
